# Converting Surfer Atlas .BNA (ASCII DAT) file to a Vector Layer

We have an ASCII file from Surfer in the [BNA format](http://surferhelp.goldensoftware.com/subsys/subsys_gsibna_hid_gsibna_filedesc.htm)  defining every building ground plan as a polygon by listing its vertices. Hence the entry for a given building is the building number followed by the number of vertices of its boundary and the rooftop elevation (assumed flat), followed by a list of the (X,Y) coordinates of each one of the vertices of its boundary. As an example the entry for building number 186 is given below:
The sequence means: Building number 105 is a polygon with 6 vertices and its rooftop elevation is 54.69 m (MSL). The (X,Y) co-ordinates of the given 6 vertices follow in the four next lines.

```
 105    6   54.69
       1651.562500      4787.500000
       1652.125000      4785.000000
       1649.062500      4787.000000
       1650.750000      4789.500000
       1653.812500      4787.500000
       1652.125000      4785.000000
 106    6   58.98
       1555.875000      4755.500000
       1558.000000      4753.000000
       1553.187500      4753.500000
       1553.687500      4757.500000
       1558.500000      4757.000000
       1558.000000      4753.000000
 107    8   62.32
       1537.062500      4741.500000
       1532.062500      4737.000000
       1532.062500      4744.500000
       1539.625000      4744.500000
       1539.437500      4742.000000
       1542.062500      4742.000000
       1541.875000      4737.000000
       1532.062500      4737.000000
```

We can creat a CSV with the polygon geometry stored as text in WKT format. QGIS can read this format easily and display the data.

In [43]:
input = 'Buildings.dat'
output = 'Buildings.csv'

In [41]:
data = []
with open(input, 'r') as f:
    for line in f:
        # Get number of verticies from the first line
        fid, numvertices, elev = line.split()
        coordinates = []
        # Skip ahead number of lines equal to number of vertices and save the coordinates
        for x in range(int(numvertices)):
            x, y = f.readline().split()
            coordinates.append(('{} {}'.format(x,y)))
        # Discard first coordinate which is the centroid
        wkt = 'POLYGON (({}))'.format(','.join(coordinates[1:]))
        data.append({'fid': int(fid), 'elev': float(elev), 'wkt': wkt})

In [42]:
import csv

with open(output, 'w') as csvfile:
    fieldnames = ['fid', 'elev', 'wkt']
    writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
    writer.writeheader()
    for row in data:
        writer.writerow(row)


The result is a CSV file that looks like this

```
fid,elev,wkt
2,127.69,"POLYGON ((627.187500 7781.000000,626.125000 7785.000000,629.062500 7786.000000,630.125000 7782.000000,627.187500 7781.000000))"
3,164.42,"POLYGON ((824.125000 7675.500000,822.687500 7679.000000,826.000000 7680.500000,827.437500 7677.000000,824.125000 7675.500000))"
4,171.19,"POLYGON ((840.125000 7640.500000,836.812500 7652.000000,842.937500 7654.000000,846.250000 7642.500000,840.125000 7640.500000))"
```

The resulting CSV can be imported using the *Delimited Text* tab in the QGIS Data Source Manager using **WKT** field as *Geometry field*

![](import_wkt.png)

The point layers loads in QGIS. Since the data also has an `elev` attribute, we can style it using the **2.5D** renderer in QGIS.
![](buildings.png)